In [6]:
import numpy as np
import matplotlib as plt

# PHYS 541 Final Project: The Cracker Barrel Peg Game
## Name: Jacob Buchanan

If you've ever been to a Cracker Barrel, you've probably made an attempt at the peg game. It's pretty straightforward. You have a triangle with some pegs and one open space to start. You have to jump all the pegs and get as few as you can at the end. It's difficult to get a good solution (I always end up doing worse the more I try). I'd like to study this game a bit by writing a simulation of it.

The triangle nature of this game makes simulating it somewhat tricky. I'll use a matrix and some defined Python classes to run this.

First I'll define a "space" class. It can hold a peg object and keeps track of activity.

In [7]:
class Space:
    
    def __init__(self, peg=None):
        self.peg = peg

I'll now define a peg. It knows how many moves it has made and if it's still in the game.

In [8]:
class Peg:
    def __init__(self):
        self.num_moves = 0
        self.in_game = True

Now I'll define the board. This is effectively a matrix with 5 rows and n columns (where n is the row number).

In [14]:
class Board:
    def __init__(self):
        self.board = [
            [Space()],
            [Space(), Space()],
            [Space(), Space(), Space()],
            [Space(), Space(), Space(), Space()],
            [Space(), Space(), Space(), Space(), Space()],
        ]

    def setup_board(self, empty_pos=(0,0)):
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                print(i, j)

board = Board()
print(board.board)
board.setup_board()

[[<__main__.Space object at 0x10ecd63d0>], [<__main__.Space object at 0x10ec97f50>, <__main__.Space object at 0x10ec97d90>], [<__main__.Space object at 0x10ec97990>, <__main__.Space object at 0x10ec97b90>, <__main__.Space object at 0x10ec97810>], [<__main__.Space object at 0x10ec97ad0>, <__main__.Space object at 0x10ec96e90>, <__main__.Space object at 0x10ec97e50>, <__main__.Space object at 0x10ec96f50>], [<__main__.Space object at 0x10ec97390>, <__main__.Space object at 0x10ec97850>, <__main__.Space object at 0x10ec97c90>, <__main__.Space object at 0x10ec97d50>, <__main__.Space object at 0x10ec961d0>]]
0 0
1 0
1 1
2 0
2 1
2 2
3 0
3 1
3 2
3 3
4 0
4 1
4 2
4 3
4 4
